# 04. 전이 학습 미니 프로젝트 : 뉴스 제목 분류하기
### 과목: NLP Transfer Learning
---
**학습 목표**
- 2번 실습(영화 리뷰)과 코드는 비슷하지만, **주제(뉴스)**와 **데이터(다중 분류)**가 다릅니다.
- 아주 작은 **미니 데이터셋**을 직접 만들어서, 전체 흐름(데이터 -> 학습 -> 예측)을 빠르게 실습해봅니다.
- **KLUE-BERT** 모델을 사용하여 **IT, 경제, 문화** 3가지 카테고리로 뉴스를 분류해봅시다.

### 1. 나만의 데이터셋 만들기 (Toy Dataset)
실습을 위해 15개 정도의 뉴스 데이터를 직접 만들어봅시다.
- **Label**: 0 (IT/과학), 1 (경제), 2 (문화)

In [ ]:
import pandas as pd

# 1. 실습용 데이터 준비 (텍스트, 정답)
data = [
    ("삼성전자, 차세대 3나노 반도체 양산 성공", 0),       # IT
    ("구글 알파고 이후 생성형 AI 시대 도래", 0),
    ("애플 비전 프로, 공간 컴퓨터의 새로운 미래", 0),
    ("카카오, AI 기술로 새로운 채팅 서비스 출시", 0),
    ("테슬라, 자율주행 소프트웨어 업데이트 발표", 0),
    
    ("코스피, 외국인 매수세에 힘입어 상승 마감", 1),      # 경제
    ("한국은행, 기준금리 3.5%로 동결 결정", 1),
    ("부동산 시장, 하반기 반등 조짐 보인다", 1),
    ("원/달러 환율, 1300원대 안정세 유지", 1),
    ("국제 유가, 중동 불안에 소폭 상승", 1),
    
    ("봉준호 감독 신작, 칸 영화제 경쟁 부문 초청", 2),   # 문화
    ("BTS 정국, 빌보드 핫100 차트 1위 석권", 2),
    ("롤드컵 결승전, 광화문 광장 거리 응원 열기", 2),
    ("국립현대미술관, 이번 주말 무료 관람 이벤트", 2),
    ("뉴진스, 신곡 뮤직비디오 유튜브 조회수 폭발", 2)
]

# 2. 데이터프레임 변환
df = pd.DataFrame(data, columns=['text', 'label'])
df.sample(5) # 랜덤하게 5개 확인

### 2. 모델 및 토크나이저 불러오기
한국어 뉴스 분류이므로 `klue/bert-base` 모델을 사용합니다.
- 우리가 풀 문제는 **3개(IT, 경제, 문화)** 중 하나를 고르는 것이므로 `num_labels=3`으로 설정합니다.

In [ ]:
from transformers import AutoTokenizer, TFBertForSequenceClassification
import tensorflow as tf

MODEL_NAME = "klue/bert-base"

# 1. 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 2. 모델 로드 (3가지 분류)
model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, from_pt=True)

print("모델 준비 완료!")

### 3. 데이터 전처리 (Tokenization)
글자를 숫자로(Input ID), 문장 길이(Attention Mask)를 맞춰주는 작업입니다.

In [ ]:
# 1. 입력 텍스트를 리스트로 변환
texts = df['text'].tolist()
labels = df['label'].tolist()

# 2. 토큰화 진행
tokenized = tokenizer(
    texts,
    padding=True,       # 문장 길이를 제일 긴 문장에 맞춤 (뒤에 0 채움)
    truncation=True,    # 너무 긴 문장은 자름
    return_tensors="tf" # 텐서플로우 형식으로 반환
)

# 3. tf.data.Dataset으로 변환 (학습용 파이프라인)
# 배치 크기는 데이터가 적으니 4로 작게 설정합니다.
dataset = tf.data.Dataset.from_tensor_slices((dict(tokenized), labels))
dataset = dataset.shuffle(buffer_size=100).batch(4)

print("데이터셋 준비 완료!")

### 4. 학습 (Fine-tuning)
아주 적은 데이터지만, BERT는 미리 공부를 많이 해왔기 때문에(Pre-trained) 금방 배웁니다.

In [ ]:
from transformers import create_optimizer

# 1. 최적화 도구(Optimizer) 설정
optimizer, _ = create_optimizer(
    init_lr=5e-5,          # 학습률
    num_train_steps=15,    # 총 학습 횟수 (데이터 15개 * 1 에폭 = 4배치 * 5에폭 정도)
    num_warmup_steps=0,
    weight_decay_rate=0.01
)

# 2. 컴파일
model.compile(
    optimizer=optimizer,
    loss=model.hf_compute_loss, # HuggingFace 전용 손실 함수
    metrics=['accuracy']        # 정확도
)

# 3. 학습 시작 (Epochs=3)
history = model.fit(dataset, epochs=3)

### 5. 실전 예측 (Inference)
내가 만든 모델이 뉴스를 잘 분류하는지 확인해봅시다.

In [ ]:
# 예측하고 싶은 뉴스 제목
new_news = [
    "엔비디아 주가 연일 최고치 경신",   # IT + 경제? (IT 쪽에 가까움)
    "블랙핑크 월드 투어 매진 행렬",     # 문화
    "물가 상승률 2%대 진입 전망"        # 경제
]

# 1. 토큰화
inputs = tokenizer(new_news, padding=True, truncation=True, return_tensors="tf")

# 2. 예측 (Logits 출력)
outputs = model(inputs)
logits = outputs.logits

# 3. 결과 해석 (가장 높은 점수의 인덱스 찾기)
predictions = tf.argmax(logits, axis=-1)

# 4. 출력
id2label = {0: "IT/과학", 1: "경제", 2: "문화"}

for i, news in enumerate(new_news):
    print(f"뉴스: {news}")
    print(f"예측: {id2label[predictions[i].numpy()]}\n")